In [1]:

from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time
import datetime
import requests
import numpy as np
import pandas as pd


In [2]:
from_date = "1/1/2022"
from_date = int(time.mktime(datetime.datetime.strptime(from_date, "%d/%m/%Y").timetuple()))
to_date = "1/1/2023"
to_date = int(time.mktime(datetime.datetime.strptime(to_date, "%d/%m/%Y").timetuple()))
interval = "1d" # other options are 1w and 1mo
tag = "GOOG"


url = f"https://finance.yahoo.com/quote/{tag}/history?period1={from_date}&period2={to_date}&interval={interval}&filter=history&frequency={interval}&includeAdjustedClose=true"

In [3]:
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)
reject_button_clickable = driver.find_element(By.CLASS_NAME, "reject-all")
ActionChains(driver)\
    .click(reject_button_clickable)\
        .perform()

time.sleep(3)
for i in range(15):        
    ActionChains(driver).scroll_by_amount(0, 1000).perform()
        
print("done")
time.sleep(5)
page_source = driver.page_source
driver.quit()
soup = BeautifulSoup(page_source, "html.parser")

done


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.198)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0104A813+48355]
	(No symbol) [0x00FDC4B1]
	(No symbol) [0x00EE5358]
	(No symbol) [0x00ECD293]
	(No symbol) [0x00F2E37B]
	(No symbol) [0x00F3C473]
	(No symbol) [0x00F2A536]
	(No symbol) [0x00F082DC]
	(No symbol) [0x00F093DD]
	GetHandleVerifier [0x012AAABD+2539405]
	GetHandleVerifier [0x012EA78F+2800735]
	GetHandleVerifier [0x012E456C+2775612]
	GetHandleVerifier [0x010D51E0+616112]
	(No symbol) [0x00FE5F8C]
	(No symbol) [0x00FE2328]
	(No symbol) [0x00FE240B]
	(No symbol) [0x00FD4FF7]
	BaseThreadInitThunk [0x764F00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77447B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77447B1E+238]


In [ ]:
table = soup.find("tbody")
table_data = list()



for row in table.find_all("tr"):
    row_data = list()
    
    for cell in row:
        invalid_row = False
        if "Stock Split" in cell.text:
            invalid_row = True 
        row_data.append(cell.text)
    if not invalid_row:
        table_data.append(row_data)
    

table_data = np.array(table_data)


In [ ]:
data_dict = pd.DataFrame({"date":table_data[:,0], "Open":table_data[:,1], "High":table_data[:,2], "Low":table_data[:,3],
                          "Close":table_data[:,4], "Volume":table_data[:,5]})

In [ ]:
data_dict

,date,Open,High,Low,Close,Volume
0,"Dec 30, 2022",87.36,88.83,87.03,88.73,88.73
1,"Dec 29, 2022",87.03,89.36,86.99,88.95,88.95
2,"Dec 28, 2022",87.50,88.52,86.37,86.46,86.46
3,"Dec 27, 2022",89.31,89.50,87.54,87.93,87.93
4,"Dec 23, 2022",87.62,90.10,87.62,89.81,89.81
...,...,...,...,...,...,...
247,"Jan 06, 2022",137.50,139.69,136.76,137.55,137.55
248,"Jan 05, 2022",144.18,144.30,137.52,137.65,137.65
249,"Jan 04, 2022",145.55,146.61,143.82,144.42,144.42
250,"Jan 03, 2022",144.48,145.55,143.50,145.07,145.07
